In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [42]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    
    embed_dim = 32  # Embedding size for each token
    vocab_size = n_word_unique + 1

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length=maxlen)
    x = embedding_layer(inputs)
    x = layers.Bidirectional(layers.SimpleRNN(128, input_dim=(None, embed_dim), 
                                         return_sequences=False, activation='tanh', 
                                         dropout=0.4))(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "rnn_models/{itr:04}/".format(itr=cnt)
    checkpoint_path = model_path + "{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir $model_path
    
    
    model.compile(tf.keras.optimizers.Adam(lr=0.001), "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    

    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_auc', verbose=1,
            save_weights_only=True, save_best_only=True, mode='max')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )

    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

mkdir: cannot create directory ‘rnn_models/0001/’: No such file or directory
Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.6924 - accuracy: 0.5381 - auc: 0.5371
Epoch 00001: val_auc improved from -inf to 0.62714, saving model to rnn_models/0001/0001.ckpt
63/63 [==============================] - 6s 91ms/step - loss: 0.6924 - accuracy: 0.5381 - auc: 0.5371 - val_loss: 0.6745 - val_accuracy: 0.5490 - val_auc: 0.6271
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.6476 - accuracy: 0.6229 - auc: 0.6742
Epoch 00002: val_auc improved from 0.62714 to 0.64711, saving model to rnn_models/0001/0002.ckpt
63/63 [==============================] - 5s 82ms/step - loss: 0.6476 - accuracy: 0.6229 - auc: 0.6742 - val_loss: 0.6583 - val_accuracy: 0.6160 - val_auc: 0.6471
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.4638 - accuracy: 0.8050 - auc: 0.8779
Epoch 00003: val_auc improved from 0.64711 to 0.69357, saving model to rnn_models/00

In [43]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))
print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))

print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))


test avg loss:  0.6931678175926208 +/- 0.08899267652347956
test avg acc:  0.7459999918937683 +/- 0.04092919284080075
test avg auc:  0.8184676051139832 +/- 0.04263245359078299


val avg loss:  0.7024811148643494 +/- 0.10407120022360954
val avg acc:  0.7526000022888184 +/- 0.03642306451623724
val avg auc:  0.813243055343628 +/- 0.04220542301297783


train avg loss:  0.013099854858592153 +/- 0.012610140689702254
train avg acc:  0.9976499915122986 +/- 0.0022380252626900787
train avg auc:  0.9998417735099793 +/- 0.000268081307113254
